### Data Dictionary

Before we proceed with analysis, let's review the data dictionary provided to understand the variables and indicators in the datasets.

Variables in kenyan_df:

    ref_area: ISO 3 country code representing geographical location/region.
    ref_area.label: Full name of the country.
    source: Source of the data.
    ... (other variables listed)

Variables in population_df:

    ccode: Country code.
    name: Name of the country.
    num_code: Numeric code.
    
Indicators:

    Inactivity rate by sex and age -- ILO modelled estimates, Nov. 2022 (%)
    Unemployment rate by sex and age -- ILO modelled estimates, Nov. 2022 (%)
    Employment by sex and economic activity -- ILO modelled estimates, Nov. 2022 (thousands)
    ... (other indicators listed)    

In [152]:
# import libraries
import pandas as pd
import numpy as np
#ignore warning
import warnings
warnings.filterwarnings('ignore')


#### Data Importing

In [153]:
# a function to read any csv file.
def read_csv_file(file_path):
    dataframe = pd.read_csv(file_path)
    return dataframe



#### Understanding Population Data

In [154]:
population_df = read_csv_file("Datasets/Population_data.csv")
population_df

,ccode,name,num_code,year,age,sex,population
0,GHA,Ghana,288,2015,15,female,184146.56
1,GHA,Ghana,288,2015,15,male,193092.96
2,GHA,Ghana,288,2015,16,female,269206.88
3,GHA,Ghana,288,2015,16,male,282046.40
4,GHA,Ghana,288,2015,17,female,323915.68
...,...,...,...,...,...,...,...
3271,RWA,Rwanda,646,2040,33,male,143553.44
3272,RWA,Rwanda,646,2040,34,female,143317.12
3273,RWA,Rwanda,646,2040,34,male,138332.32
3274,RWA,Rwanda,646,2040,35,female,138679.04


In [155]:
#check for column names, dtypes and missing values
print(population_df.shape) # prints the dimension of the df
population_df.info()

(3276, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3276 entries, 0 to 3275
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ccode       3276 non-null   object 
 1   name        3276 non-null   object 
 2   num_code    3276 non-null   int64  
 3   year        3276 non-null   int64  
 4   age         3276 non-null   int64  
 5   sex         3276 non-null   object 
 6   population  3276 non-null   float64
dtypes: float64(1), int64(3), object(3)
memory usage: 179.3+ KB


Based on the provided information about the columns in a dataset:
- The dataset contains information for `3,276` entries.
- The dataset seems to contain demographic information, possibly related to different countries or regions, given the presence of codes, names, and population counts.
- The dataset's memory usage is approximately `179.3 KB`.
- Since all columns have non-null values for all entries, there don't appear to be any missing values in the dataset.
- The dataset appears to have a mix of categorical and numerical data, with information about population, age, and sex, among other variables.
- Without more context or specific analysis, it's not possible to draw more detailed conclusions about the dataset's content or purpose.

For now we filter for the **Kenyan youth population the year 2015-2024**, which will act as our base for analysis

In [156]:
# a function that filters for the kenyan populaion, the youth bracket and for the years 2015-2024
def process_population_data(population, country_code):
    # Step 1: Filter population data for Kenya
    kenya_pop = population[population['ccode'] == country_code]
    
    # Step 2: Create age groups using bins and labels
    bins = [14, 24, 35]
    labels = ['15-24', '25-35']
    kenya_pop['age_group'] = pd.cut(kenya_pop['age'], bins=bins, labels=labels, right=True)
    
    # Step 3: Group by name, age group, sex, and year to sum up population
    grouped_pop_df = kenya_pop.groupby(['name', 'age_group', 'sex', 'year'])['population'].sum().reset_index()
    
    # Step 4: Filter years to include only 2015-2024
    years_to_filter = list(range(2015, 2025))
    filtered_df = grouped_pop_df[grouped_pop_df['year'].isin(years_to_filter)]
    
    return filtered_df

# Country code for Kenya
kenya_country_code = 'KEN'

# Call the function to process population data for Kenya
filtered_population = process_population_data(population_df, kenya_country_code)

# Display the first few rows of the processed data
filtered_population.head()


,name,age_group,sex,year,population
0,Kenya,15-24,female,2015,4742700.0
1,Kenya,15-24,female,2016,4887980.0
2,Kenya,15-24,female,2017,5033260.0
3,Kenya,15-24,female,2018,5178540.0
4,Kenya,15-24,female,2019,5323820.0


In [157]:
filtered_population.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 0 to 87
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   name        40 non-null     object  
 1   age_group   40 non-null     category
 2   sex         40 non-null     object  
 3   year        40 non-null     int64   
 4   population  40 non-null     float64 
dtypes: category(1), float64(1), int64(1), object(2)
memory usage: 1.7+ KB


Our data size has been reduced to `40` entries from the previous one. 

#### Understanding the Kenyan_National Data

In [158]:
#import the kenyan dataset
kenyan_df = read_csv_file('Datasets/Kenya_National.csv')
#check the columns, missing values and dtypes
print(kenyan_df.info())

#prints the first few rows
kenyan_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11869 entries, 0 to 11868
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ref_area              11869 non-null  object 
 1   ref_area.label        11869 non-null  object 
 2   source                11869 non-null  object 
 3   source.label          11869 non-null  object 
 4   indicator             11869 non-null  object 
 5   indicator.label       11869 non-null  object 
 6   sex                   11869 non-null  object 
 7   sex.label             11869 non-null  object 
 8   classif1              11869 non-null  object 
 9   classif1.label        11869 non-null  object 
 10  classif2              9703 non-null   object 
 11  classif2.label        9703 non-null   object 
 12  time                  11869 non-null  int64  
 13  obs_value             11278 non-null  float64
 14  obs_status            5816 non-null   object 
 15  obs_status.label   

,ref_area,ref_area.label,source,source.label,indicator,indicator.label,sex,sex.label,classif1,classif1.label,...,time,obs_value,obs_status,obs_status.label,note_classif,note_classif.label,note_indicator,note_indicator.label,note_source,note_source.label
0,KEN,Kenya,XA:1909,ILO - Modelled Estimates,EMP_2EMP_SEX_ECO_NB,Employment by sex and economic activity -- ILO...,SEX_T,Sex: Total,ECO_SECTOR_TOTAL,Economic activity (Broad sector): Total,...,2021,22755.313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KEN,Kenya,XA:1909,ILO - Modelled Estimates,EMP_2EMP_SEX_ECO_NB,Employment by sex and economic activity -- ILO...,SEX_T,Sex: Total,ECO_SECTOR_AGR,Economic activity (Broad sector): Agriculture,...,2021,7516.948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KEN,Kenya,XA:1909,ILO - Modelled Estimates,EMP_2EMP_SEX_ECO_NB,Employment by sex and economic activity -- ILO...,SEX_T,Sex: Total,ECO_SECTOR_IND,Economic activity (Broad sector): Industry,...,2021,3578.747,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,KEN,Kenya,XA:1909,ILO - Modelled Estimates,EMP_2EMP_SEX_ECO_NB,Employment by sex and economic activity -- ILO...,SEX_T,Sex: Total,ECO_SECTOR_SER,Economic activity (Broad sector): Services,...,2021,11659.617,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KEN,Kenya,XA:1909,ILO - Modelled Estimates,EMP_2EMP_SEX_ECO_NB,Employment by sex and economic activity -- ILO...,SEX_M,Sex: Male,ECO_SECTOR_TOTAL,Economic activity (Broad sector): Total,...,2021,11502.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The DataFrame contains `11,869` entries.
There are `22` columns in the DataFrame, each with specific data:
The DataFrame includes a mix of categorical and numerical data, with various kinds of labels and notes associated with different categories.
- There are missing values in columns such as `"classif2,"` `"obs_value,"` `"obs_status,"` `"note_classif,"` `"note_indicator,"` and `"note_source,"` which might need to be handled depending on the analysis or use case.
- The data seems to be organized around observations of different indicators over time, with various dimensions like reference area, source, sex, and classification categories.
 The `"time"` column likely represents the time period in which the observations were made, and the `"obs_value"` column represents the numerical values of the observed data.
- The columns with `".label"` in their names seem to provide labels associated with the corresponding categorical columns, possibly providing more descriptive information.
- The presence of note-related columns suggests additional textual information associated with the data, possibly explaining classifications, indicators, and sources.

 #### 2. Apply Inactivity Rate by Age and Gender

We'll apply the ILO inactive rate by age and gender to the corresponding population for 2015 - 2024 to get the total inactive population.

In [159]:
def process_kenya_data(data, years_to_filter):
    # Step 1: Filtering the data for specific age groups
    data = data[data['classif1.label'].isin(['Age (Youth, adults): 15-24', 'Age (Youth, adults): 25+'])]
    
    # Step 2: Further filtering for specific genders
    data = data[data['sex.label'].isin(['Sex: Male', 'Sex: Female'])]

    # Step 3: Cleaning and standardizing age group labels
    data['classif1.label'] = data['classif1.label'].str.replace('Age (Youth, adults): 25+', '25-35', regex=False).str.strip()
    data['classif1.label'] = data['classif1.label'].str.replace('Age (Youth, adults): 15-24', '15-24', regex=False).str.strip()

    # Step 4: Cleaning and standardizing gender labels
    data['sex.label'] = data['sex.label'].str.replace('Sex: Male', 'male', regex=False).str.strip()
    data['sex.label'] = data['sex.label'].str.replace('Sex: Female', 'female', regex=False).str.strip()

    # Step 5: Filtering data based on specified years
    data = data[data['time'].isin(years_to_filter)]
    
    return data
# Example usage:
years_to_filter =  list(range(2015,2025)) 
filtered_kenyan_df = process_kenya_data(kenyan_df, years_to_filter)
#print(filtered_kenyan_df.info())

In [160]:
def calculate_inactive_population(data, data2):
    # Step 6: Filtering data for a specific indicator
    inactivity_sex_age = data[data['indicator.label'] == 'Inactivity rate by sex and age -- ILO modelled estimates, Nov. 2022 (%)']

    # Step 7: Creating a mapping dictionary based on specific columns
    obs_value_map = inactivity_sex_age.set_index(['classif1.label', 'time', 'sex.label'])['obs_value'].to_dict()

    # Step 8: Mapping values to calculate 'ILO inactive share'
    data2['ILO_inactive_share'] = data2.set_index(['age_group', 'year', 'sex']).index.map(obs_value_map).astype(float)
    
    # Step 9: Calculating 'Total inactive population'
    data2['total_inactive_population'] = data2['population'] * data2['ILO_inactive_share'] / 100

    # Step 10: Returning the processed DataFrame
    return data2

# Example usage:
filtered_inactive_pop = calculate_inactive_population(filtered_kenyan_df, filtered_population)
filtered_inactive_pop.head()

,name,age_group,sex,year,population,ILO_inactive_share,total_inactive_population
0,Kenya,15-24,female,2015,4742700.0,55.928,2.652497e+06
1,Kenya,15-24,female,2016,4887980.0,56.361,2.754914e+06
2,Kenya,15-24,female,2017,5033260.0,56.906,2.864227e+06
3,Kenya,15-24,female,2018,5178540.0,57.463,2.975744e+06
4,Kenya,15-24,female,2019,5323820.0,58.043,3.090105e+06


In [161]:
# checking for null values
filtered_inactive_pop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 0 to 87
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   name                       40 non-null     object  
 1   age_group                  40 non-null     category
 2   sex                        40 non-null     object  
 3   year                       40 non-null     int64   
 4   population                 40 non-null     float64 
 5   ILO_inactive_share         40 non-null     float64 
 6   total_inactive_population  40 non-null     float64 
dtypes: category(1), float64(3), int64(1), object(2)
memory usage: 2.3+ KB


After the imputations, more columns have been added in the populations data with no missing values 

#### 3: Apply Unemployment Rate by Age and Gender

We'll apply the ILO unemployment rate by age and gender to the corresponding population for 2015 - 2024 to get the total unemployed population.

In [162]:
def calculate_unemployment_data(data, data1):
    # Step 1: Filter data for the specific indicator
    unemployment_sex_age = data[data['indicator.label'] == 'Unemployment rate by sex and age -- ILO modelled estimates, Nov. 2022 (%)']
    
    # Step 2: Create a dictionary to map combination of attributes to 'obs_value'
    obs_value_map = unemployment_sex_age.set_index(['classif1.label', 'time', 'sex.label'])['obs_value'].to_dict()

    # Step 3: Map values to create 'ILO unemployed rate' column
    data1['ILO_unemployed_rate'] = data1.set_index(['age_group', 'year', 'sex']).index.map(obs_value_map)

    # Step 4: Calculate 'Total unemployed population' based on 'ILO unemployed rate', population, and inactive population
    data1['total_unemployed_population'] = data1['ILO_unemployed_rate'].astype(float) / 100 * (data1['population'] - data1['total_inactive_population'])

    # Step 5: Returning the updated DataFrame
    return data1

# usage:
filtered_unemployed_pop = calculate_unemployment_data(filtered_kenyan_df, filtered_inactive_pop)
filtered_unemployed_pop.head()

,name,age_group,sex,year,population,ILO_inactive_share,total_inactive_population,ILO_unemployed_rate,total_unemployed_population
0,Kenya,15-24,female,2015,4742700.0,55.928,2.652497e+06,7.339,153399.979382
1,Kenya,15-24,female,2016,4887980.0,56.361,2.754914e+06,7.371,157228.264801
2,Kenya,15-24,female,2017,5033260.0,56.906,2.864227e+06,9.304,201806.836312
3,Kenya,15-24,female,2018,5178540.0,57.463,2.975744e+06,11.192,246536.879053
4,Kenya,15-24,female,2019,5323820.0,58.043,3.090105e+06,13.175,294291.971987


#### 3. Get Total Employed Population:

Calculate the employed population for each year, age group, and gender by subtracting the inactive and unemployed populations from the total population.

In [163]:
# Calculate the total employed population by subtracting the sum of total inactive and total unemployed populations from the total population.
filtered_unemployed_pop['total_employed_population'] = filtered_unemployed_pop['population'] - (filtered_unemployed_pop['total_inactive_population'] + filtered_unemployed_pop['total_unemployed_population'])

# Display the first few rows of the DataFrame with the newly calculated 'total_employed_population' column.
filtered_unemployed_pop.head()


,name,age_group,sex,year,population,ILO_inactive_share,total_inactive_population,ILO_unemployed_rate,total_unemployed_population,total_employed_population
0,Kenya,15-24,female,2015,4742700.0,55.928,2.652497e+06,7.339,153399.979382,1.936803e+06
1,Kenya,15-24,female,2016,4887980.0,56.361,2.754914e+06,7.371,157228.264801,1.975837e+06
2,Kenya,15-24,female,2017,5033260.0,56.906,2.864227e+06,9.304,201806.836312,1.967226e+06
3,Kenya,15-24,female,2018,5178540.0,57.463,2.975744e+06,11.192,246536.879053,1.956259e+06
4,Kenya,15-24,female,2019,5323820.0,58.043,3.090105e+06,13.175,294291.971987,1.939423e+06


#### 4. Apply Shares of ISIC REV 4 Sector to Employed Population:
- Calculate the shares for each sector by dividing the obs_value for each sector by the total obs_value for all sectors in the same year and age group.
- Multiply these shares by the employed population to get absolute numbers for each sector.
- Merge this data with the kenyan_df DataFrame.

We takes a dataset containing employment information as input.
We filters the data to focus on a specified type of indicator related to employment and specified sectors of economic activity.Then we convert the values in a column called 'obs_value' to numbers (if possible) and handles any errors that may occur during the conversion. Later we groups the filtered data by different categories: sector of `economic activity`, `gender`, and `year`.
We then calculate the sum of the `'obs_value'` column for each group created in the previous step.
Lastly we creates a new summarized dataset with the grouped information.
*NB* _The summarized dataset is returned as the output of the function_

In [164]:
def process_employment_data(data):
    # Filter by indicator and ISIC REV 4 sectors
    employment_sex_age_economic = data[data['indicator.label'] == 'Employment by sex, age and economic activity (thousands)']
    employment_sex_age_economic = employment_sex_age_economic[employment_sex_age_economic['classif2.label'].str.contains('ISIC-Rev.4')]

    # Convert 'obs_value' column to numeric
    employment_sex_age_economic['obs_value'] = pd.to_numeric(employment_sex_age_economic['obs_value'], errors='coerce')

    # Group by sector, sex, and year and sum the 'obs_value'
    sector_shares_not_modelled = employment_sex_age_economic.groupby(['classif2.label', 'sex.label', 'time'])['obs_value'].sum().reset_index()

    return sector_shares_not_modelled

sector_shares_not_modelled = process_employment_data(filtered_kenyan_df)
sector_shares_not_modelled.head()


,classif2.label,sex.label,time,obs_value
0,Economic activity (ISIC-Rev.4): A. Agriculture...,female,2019,3192.079
1,Economic activity (ISIC-Rev.4): A. Agriculture...,male,2019,3096.289
2,Economic activity (ISIC-Rev.4): B. Mining and ...,female,2019,59.451
3,Economic activity (ISIC-Rev.4): B. Mining and ...,male,2019,102.193
4,Economic activity (ISIC-Rev.4): C. Manufacturing,female,2019,549.498


The  below function calculates the sector shares within a dataset of not modeled employment information. 
It starts by calculating the total values for each year and gender in the dataset. It filters rows where the sector is labeled as `"Total"` and calculates the sum of the 'obs_value' column for each group of gender and year. This helps in later calculations to find the proportion of each sector's value within the total for that year and gender.
Then, using vectorized operations (operations applied to entire arrays/columns rather than individual elements), it iterates through each unique year and gender combination in the `sector_shares_not_modelled` DataFrame.

In [165]:
def calculate_sector_shares(sector_shares_not_modelled):
    # Calculate total values for each year
    total_values = sector_shares_not_modelled[sector_shares_not_modelled["classif2.label"].str.contains("Total")]
    total_values = total_values.groupby(["sex.label", "time"])["obs_value"].sum().reset_index()

    # Calculate sector shares using vectorized operations
    for year in sector_shares_not_modelled["time"].unique():
        for gender in sector_shares_not_modelled["sex.label"].unique():
            year_gender_rows = sector_shares_not_modelled[(sector_shares_not_modelled["time"] == year) & (sector_shares_not_modelled["sex.label"] == gender)]
            sector_total = total_values[(total_values["time"] == year) & (total_values["sex.label"] == gender)]["obs_value"].values[0]
            sector_shares_not_modelled.loc[year_gender_rows.index, "sector_shares"] = year_gender_rows["obs_value"] / sector_total
    
    return sector_shares_not_modelled

#display our not modelled df
sector_shares_not_modelled = calculate_sector_shares(sector_shares_not_modelled)
sector_shares_not_modelled.head()

,classif2.label,sex.label,time,obs_value,sector_shares
0,Economic activity (ISIC-Rev.4): A. Agriculture...,female,2019,3192.079,0.357879
1,Economic activity (ISIC-Rev.4): A. Agriculture...,male,2019,3096.289,0.315783
2,Economic activity (ISIC-Rev.4): B. Mining and ...,female,2019,59.451,0.006665
3,Economic activity (ISIC-Rev.4): B. Mining and ...,male,2019,102.193,0.010422
4,Economic activity (ISIC-Rev.4): C. Manufacturing,female,2019,549.498,0.061607


In [166]:
sectors = ['Agriculture','Mining and quarrying','Manufacturing','Electricity','Water supply','Construction','Wholesale and retail',
           'Transport', 'Accommodation','Information and communication','Financial and insurance','Real estate',
           'Professional','Administrative and support','Public administration and defence','Education','Human health','Arts $ entertainment',
           'Other services','Activities of households','Extraterritorial organizations']

def calculate_and_merge_sector_shares(filtered_unemployed_pop, sector_shares_not_modelled, sectors):
    # Initialize columns in the filtered_unemployed_pop DataFrame with zeros for each sector
    for column in sectors:
        filtered_unemployed_pop[column] = 0
        
    # Rename columns in sector_shares_not_modelled DataFrame
    new_column_names = {'sex.label': 'sex','time':'year'}
    sector_shares_not_modelled = sector_shares_not_modelled.rename(columns=new_column_names)

    # Iterate through each sector
    for i in sectors:
        # Iterate through unique years in sector_shares_not_modelled
        for year in sector_shares_not_modelled["year"].unique():
            # Iterate through unique genders in sector_shares_not_modelled
            for gender in sector_shares_not_modelled["sex"].unique():
                # Define conditions to select rows matching sector, gender, and year
                conditions = (sector_shares_not_modelled['classif2.label'].str.contains(i)) & \
                             (sector_shares_not_modelled['sex'] == gender) & \
                             (sector_shares_not_modelled['year'] == year)

                # Select rows from sector_shares_not_modelled based on conditions
                selected_rows = sector_shares_not_modelled.loc[conditions]

                # Check if selected_rows is not empty
                if not selected_rows.empty:
                    # Get the calculated sector share for the selected rows
                    selected_sector_shares = selected_rows['sector_shares'].iloc[0]

                    # Define conditions to select rows in filtered_unemployed_pop
                    population_conditions = (filtered_unemployed_pop['sex'] == gender) & \
                                            (filtered_unemployed_pop['year'] == year)

                    # Update the sector column in filtered_unemployed_pop with calculated sector shares
                    filtered_unemployed_pop.loc[population_conditions, i] = selected_sector_shares

    return filtered_unemployed_pop

population_sectors = calculate_and_merge_sector_shares(filtered_unemployed_pop, sector_shares_not_modelled, sectors)
population_sectors.head()

,name,age_group,sex,year,population,ILO_inactive_share,total_inactive_population,ILO_unemployed_rate,total_unemployed_population,total_employed_population,...,Real estate,Professional,Administrative and support,Public administration and defence,Education,Human health,Arts $ entertainment,Other services,Activities of households,Extraterritorial organizations
0,Kenya,15-24,female,2015,4742700.0,55.928,2.652497e+06,7.339,153399.979382,1.936803e+06,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0,0,0.000000,0
1,Kenya,15-24,female,2016,4887980.0,56.361,2.754914e+06,7.371,157228.264801,1.975837e+06,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0,0,0.000000,0
2,Kenya,15-24,female,2017,5033260.0,56.906,2.864227e+06,9.304,201806.836312,1.967226e+06,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0,0,0.000000,0
3,Kenya,15-24,female,2018,5178540.0,57.463,2.975744e+06,11.192,246536.879053,1.956259e+06,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0,0,0.000000,0
4,Kenya,15-24,female,2019,5323820.0,58.043,3.090105e+06,13.175,294291.971987,1.939423e+06,...,0.002669,0.004663,0.012649,0.018846,0.06666,0.020684,0,0,0.049208,0


Now we perform a calculation to determine the total employed population within each sector. We iterate through each row in the population_sectors DataFrame, which contains information about the distribution of population across different economic sectors. The objective is to calculate the total number of employed individuals within each sector.

In [167]:
# Total employed population per sector
for index, row in population_sectors.iterrows():
    for col in sectors:
        population_sectors.at[index, col] = row[col] * row['total_employed_population']

population_sectors.head()

,name,age_group,sex,year,population,ILO_inactive_share,total_inactive_population,ILO_unemployed_rate,total_unemployed_population,total_employed_population,...,Real estate,Professional,Administrative and support,Public administration and defence,Education,Human health,Arts $ entertainment,Other services,Activities of households,Extraterritorial organizations
0,Kenya,15-24,female,2015,4742700.0,55.928,2.652497e+06,7.339,153399.979382,1.936803e+06,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0
1,Kenya,15-24,female,2016,4887980.0,56.361,2.754914e+06,7.371,157228.264801,1.975837e+06,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0
2,Kenya,15-24,female,2017,5033260.0,56.906,2.864227e+06,9.304,201806.836312,1.967226e+06,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0
3,Kenya,15-24,female,2018,5178540.0,57.463,2.975744e+06,11.192,246536.879053,1.956259e+06,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0
4,Kenya,15-24,female,2019,5323820.0,58.043,3.090105e+06,13.175,294291.971987,1.939423e+06,...,5175.457976,9044.1139,24532.436187,36550.883741,129282.731494,40115.126542,0,0,95435.688604,0


#### 5.  Get Sector Growth Rates:

Calculate the sector growth rates based on the change in shares from the modeled dataset to fill in missing years.

In [168]:
def process_employment_data(df, years_to_filter):
    # Filter by indicator and relevant columns
    employment_sex_economic = df[df['indicator.label'] == 'Employment by sex and economic activity -- ILO modelled estimates, Nov. 2022 (thousands)']
    employment_sex_economic = employment_sex_economic[employment_sex_economic['time'].isin(years_to_filter)]
    employment_sex_economic = employment_sex_economic[employment_sex_economic['sex.label'].isin(['Sex: Male','Sex: Female'])]
    employment_sex_economic['sex.label'] = employment_sex_economic['sex.label'].str.replace('Sex: Male', 'male', regex=False).str.strip()
    employment_sex_economic['sex.label'] = employment_sex_economic['sex.label'].str.replace('Sex: Female', 'female', regex=False).str.strip()
    employment_sex_economic = employment_sex_economic[employment_sex_economic['classif1.label'].str.contains('Detailed')]

    # Group by classif1.label, sex.label, and time, and sum obs_value
    sector_shares_ILO = employment_sex_economic.groupby(['classif1.label', 'sex.label', 'time'])['obs_value'].sum().reset_index()
    
    return sector_shares_ILO

# Usage
sector_shares_ILO = process_employment_data(kenyan_df, years_to_filter)
sector_shares_ILO.head()

,classif1.label,sex.label,time,obs_value
0,Economic activity (Detailed): Accommodation an...,female,2015,370.713
1,Economic activity (Detailed): Accommodation an...,female,2016,401.968
2,Economic activity (Detailed): Accommodation an...,female,2017,429.887
3,Economic activity (Detailed): Accommodation an...,female,2018,464.610
4,Economic activity (Detailed): Accommodation an...,female,2019,498.249


We now calculate sector shares for each sector, gender, and year by dividing the observed values by the corresponding total values. Then we rename columns and return the updated DataFrame containing the calculated sector shares and renamed columns.

In [169]:
def calculate_and_rename_sector_shares(df):
    # Calculate total values for each year
    total_values = df[sector_shares_ILO["classif1.label"].str.contains("Total")]
    total_values = total_values.groupby(["sex.label", "time"])["obs_value"].sum().reset_index()

    # Iterate through unique years and genders
    for year in df["time"].unique():
        for gender in df["sex.label"].unique():
            # Filter sector rows for the current year and gender
            year_gender_rows = df[(df["time"] == year) & (df["sex.label"] == gender)]
            # Calculate sector_total for the current year and gender
            sector_total = total_values[(total_values["time"] == year) & (total_values["sex.label"] == gender)]["obs_value"].values[0]
            # Calculate sector shares using vectorized operations
            df.loc[year_gender_rows.index, "sector_shares"] = year_gender_rows["obs_value"].astype(float) / sector_total
    
    # Rename columns using the rename() method
    new_column_names = {'sex.label': 'sex', 'time': 'year'}
    df = df.rename(columns=new_column_names)
    
    return df

# Usage
sector_shares_ILO = calculate_and_rename_sector_shares(sector_shares_ILO)
sector_shares_ILO.head()


,classif1.label,sex,year,obs_value,sector_shares
0,Economic activity (Detailed): Accommodation an...,female,2015,370.713,0.079383
1,Economic activity (Detailed): Accommodation an...,female,2016,401.968,0.083459
2,Economic activity (Detailed): Accommodation an...,female,2017,429.887,0.086636
3,Economic activity (Detailed): Accommodation an...,female,2018,464.610,0.090920
4,Economic activity (Detailed): Accommodation an...,female,2019,498.249,0.094740


In [170]:
# Calculate growth rates for each sector, gender, and year
growth_rates = {}

for sector in sector_shares_ILO['classif1.label'].unique():
  for gender in sector_shares_ILO['sex'].unique():
    sector_gender_data = sector_shares_ILO[(sector_shares_ILO['classif1.label'] == sector) & (sector_shares_ILO['sex'] == gender)]
    sector_gender_data = sector_gender_data.sort_values(by='year') #Ensure data is sorted by year
    for i in range(1, len(sector_gender_data)):
      old_value = sector_gender_data.iloc[i - 1]['sector_shares']
      new_value = sector_gender_data.iloc[i]['sector_shares']
      year = sector_gender_data.iloc[i]['year']

      growth_rate = ((new_value - old_value) / old_value) * 100
      growth_rates[(sector, gender, year)] = growth_rate

# Now growth_rates contains the calculated growth rates for each sector, gender, and year
# Creating lists to store the data
economic_activity = []
gender = []
year = []
value = []

# Parsing the data dictionary and extracting values
for key, val in growth_rates.items():
    economic_activity.append(key[0])
    gender.append(key[1])
    year.append(key[2])
    value.append(val)

# Creating a DataFrame
sector_growth_rates = pd.DataFrame({
    'economic_activity': economic_activity,
    'sex': gender,
    'year': year,
    'growth_rate': value
})

sector_growth_rates

,economic_activity,sex,year,growth_rate
0,Economic activity (Detailed): Accommodation an...,female,2016,5.134382
1,Economic activity (Detailed): Accommodation an...,female,2017,3.806893
2,Economic activity (Detailed): Accommodation an...,female,2018,4.945109
3,Economic activity (Detailed): Accommodation an...,female,2019,4.201077
4,Economic activity (Detailed): Accommodation an...,female,2020,-2.412885
...,...,...,...,...
175,Economic activity (Detailed): Wholesale and re...,male,2017,-2.581326
176,Economic activity (Detailed): Wholesale and re...,male,2018,-1.106565
177,Economic activity (Detailed): Wholesale and re...,male,2019,-0.176497
178,Economic activity (Detailed): Wholesale and re...,male,2020,-0.914661


In [171]:
def calculate_sector_growth_rates(df):
    # Initialize an empty dictionary to store growth rates
    growth_rates = {}

    # Iterate through unique sectors and genders
    for sector in df['classif1.label'].unique():
        for gender in df['sex'].unique():
            # Filter data for the current sector and gender
            sector_gender_data = df[(sector_shares_ILO['classif1.label'] == sector) & (df['sex'] == gender)]
            # Sort data by year to ensure it's in chronological order
            sector_gender_data = sector_gender_data.sort_values(by='year')
            
            # Iterate through data rows, starting from the second row
            for i in range(1, len(sector_gender_data)):
                old_value = sector_gender_data.iloc[i - 1]['sector_shares']
                new_value = sector_gender_data.iloc[i]['sector_shares']
                year = sector_gender_data.iloc[i]['year']

                # Calculate growth rate and store it in the dictionary
                growth_rate = ((new_value - old_value) / old_value) * 100
                growth_rates[(sector, gender, year)] = growth_rate

    # Create lists to store parsed data
    economic_activity = []
    gender = []
    year = []
    value = []

    # Parse the growth_rates dictionary and extract values
    for key, val in growth_rates.items():
        economic_activity.append(key[0])
        gender.append(key[1])
        year.append(key[2])
        value.append(val)

    # Create a DataFrame to store sector growth rates
    sector_growth_rates = pd.DataFrame({
        'economic_activity': economic_activity,
        'sex': gender,
        'year': year,
        'growth_rate': value
    })

    return sector_growth_rates

# Usage
sector_growth_rates = calculate_sector_growth_rates(sector_shares_ILO)
sector_growth_rates.head()


,economic_activity,sex,year,growth_rate
0,Economic activity (Detailed): Accommodation an...,female,2016,5.134382
1,Economic activity (Detailed): Accommodation an...,female,2017,3.806893
2,Economic activity (Detailed): Accommodation an...,female,2018,4.945109
3,Economic activity (Detailed): Accommodation an...,female,2019,4.201077
4,Economic activity (Detailed): Accommodation an...,female,2020,-2.412885


Let's copy our dataframe so that we do not tamper with it

In [172]:
pop_df = population_sectors.copy()
pop_df.head()

,name,age_group,sex,year,population,ILO_inactive_share,total_inactive_population,ILO_unemployed_rate,total_unemployed_population,total_employed_population,...,Real estate,Professional,Administrative and support,Public administration and defence,Education,Human health,Arts $ entertainment,Other services,Activities of households,Extraterritorial organizations
0,Kenya,15-24,female,2015,4742700.0,55.928,2.652497e+06,7.339,153399.979382,1.936803e+06,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0
1,Kenya,15-24,female,2016,4887980.0,56.361,2.754914e+06,7.371,157228.264801,1.975837e+06,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0
2,Kenya,15-24,female,2017,5033260.0,56.906,2.864227e+06,9.304,201806.836312,1.967226e+06,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0
3,Kenya,15-24,female,2018,5178540.0,57.463,2.975744e+06,11.192,246536.879053,1.956259e+06,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0
4,Kenya,15-24,female,2019,5323820.0,58.043,3.090105e+06,13.175,294291.971987,1.939423e+06,...,5175.457976,9044.1139,24532.436187,36550.883741,129282.731494,40115.126542,0,0,95435.688604,0


In [173]:
# List of columns with null values that need imputation
sectors = ['Accommodation', 'Agriculture', 'Construction', 'Education', 'Financial and insurance', 'Human health', 'Manufacturing',
           'Mining and quarrying', 'Other services', 'Public administration and defence', 'Real estate', 'Transport', 'Utilities', 'Wholesale and retail']

# Iterate through each row in the pop_df dataframe
for index, row in pop_df.iterrows():
    if row['year'] != 2019:  # Exclude the base year (2019)
        for sector in sectors:
            prev_year = row['year'] - 1
            growth_rate_row = sector_growth_rates[(sector_growth_rates['economic_activity'].str.contains(sector)) &
                                                  (sector_growth_rates['sex'] == row['sex']) &
                                                  (sector_growth_rates['year'] == row['year'])]

            # Check if the growth_rate_row is empty
            if not growth_rate_row.empty:
              # Retrieve the growth rate from the growth_rate_row
              growth_rate = growth_rate_row['growth_rate'].values[0]

                # Find the base value from 2019 for the current sector, sex, and age group
              if sector == 'Utilities':
                utility_sectors = ['Water supply','Electricity']
                for sector in utility_sectors:
                  base_value = pop_df.loc[(pop_df['year'] == 2019) &
                                    (pop_df['sex'] == row['sex']) &
                                    (pop_df['age_group'] == row['age_group']), sector].values[0]

                    # Calculate the imputed value based on the rules you mentioned
                  if row['year'] < 2019:
                    imputed_value = base_value * (100 - growth_rate)
                  else:
                    imputed_value = base_value * (100 + growth_rate)

                    # Update the imputed value in the pop_df dataframe
                  pop_df.at[index, sector] = imputed_value

              if sector == 'Transport':
                transport_sectors = ['Transport','Information and communication']
                for sector in transport_sectors:
                  base_value = pop_df.loc[(pop_df['year'] == 2019) &
                                      (pop_df['sex'] == row['sex']) &
                                      (pop_df['age_group'] == row['age_group']), sector].values[0]

                  # Calculate the imputed value based on the rules you mentioned
                  if row['year'] < 2019:
                      imputed_value = base_value * (100 - growth_rate)
                  else:
                      imputed_value = base_value * (100 + growth_rate)

                  # Update the imputed value in the pop_df dataframe
                  pop_df.at[index, sector] = imputed_value

              if sector == 'Real estate':
                realestate_sectors = ['Real estate','Professional','Administrative and support']
                for sector in realestate_sectors:
                  base_value = pop_df.loc[(pop_df['year'] == 2019) &
                                      (pop_df['sex'] == row['sex']) &
                                      (pop_df['age_group'] == row['age_group']), sector].values[0]

                  # Calculate the imputed value based on the rules you mentioned
                  if row['year'] < 2019:
                      imputed_value = base_value * (100 - growth_rate)
                  else:
                      imputed_value = base_value * (100 + growth_rate)

                  # Update the imputed value in the pop_df dataframe
                  pop_df.at[index, sector] = imputed_value

              if sector == 'Other services':
                otherservices_sectors = ['Arts $ entertainment','Other services','Activities of households','Extraterritorial organizations']
                for sector in otherservices_sectors:
                  base_value = pop_df.loc[(pop_df['year'] == 2019) &
                                      (pop_df['sex'] == row['sex']) &
                                      (pop_df['age_group'] == row['age_group']), sector].values[0]

                  # Calculate the imputed value based on the rules you mentioned
                  if row['year'] < 2019:
                      imputed_value = base_value * (100 - growth_rate)
                  else:
                      imputed_value = base_value * (100 + growth_rate)

                  # Update the imputed value in the pop_df dataframe
                  pop_df.at[index, sector] = imputed_value

              base_value = pop_df.loc[(pop_df['year'] == 2019) &
                                (pop_df['sex'] == row['sex']) &
                                (pop_df['age_group'] == row['age_group']), sector].values[0]

              # Calculate the imputed value based on the rules you mentioned
              if row['year'] < 2019:
                  imputed_value = base_value * (100 - growth_rate)
              else:
                  imputed_value = base_value * (100 + growth_rate)

              # Update the imputed value in the pop_df dataframe
              pop_df.at[index, sector] = imputed_value
            else:
                # Handle the case when growth_rate_row is empty (no matching data found)
                print(f"No growth rate data found for sector '{sector}' and year {row['year']}")
pop_df.head()


No growth rate data found for sector 'Accommodation' and year 2015
No growth rate data found for sector 'Agriculture' and year 2015
No growth rate data found for sector 'Construction' and year 2015
No growth rate data found for sector 'Education' and year 2015
No growth rate data found for sector 'Financial and insurance' and year 2015
No growth rate data found for sector 'Human health' and year 2015
No growth rate data found for sector 'Manufacturing' and year 2015
No growth rate data found for sector 'Mining and quarrying' and year 2015
No growth rate data found for sector 'Other services' and year 2015
No growth rate data found for sector 'Public administration and defence' and year 2015
No growth rate data found for sector 'Real estate' and year 2015
No growth rate data found for sector 'Transport' and year 2015
No growth rate data found for sector 'Utilities' and year 2015
No growth rate data found for sector 'Wholesale and retail' and year 2015
No growth rate data found for secto

No growth rate data found for sector 'Accommodation' and year 2022
No growth rate data found for sector 'Agriculture' and year 2022
No growth rate data found for sector 'Construction' and year 2022
No growth rate data found for sector 'Education' and year 2022
No growth rate data found for sector 'Financial and insurance' and year 2022
No growth rate data found for sector 'Human health' and year 2022
No growth rate data found for sector 'Manufacturing' and year 2022
No growth rate data found for sector 'Mining and quarrying' and year 2022
No growth rate data found for sector 'Other services' and year 2022
No growth rate data found for sector 'Public administration and defence' and year 2022
No growth rate data found for sector 'Real estate' and year 2022
No growth rate data found for sector 'Transport' and year 2022
No growth rate data found for sector 'Utilities' and year 2022
No growth rate data found for sector 'Wholesale and retail' and year 2022
No growth rate data found for secto

,name,age_group,sex,year,population,ILO_inactive_share,total_inactive_population,ILO_unemployed_rate,total_unemployed_population,total_employed_population,...,Real estate,Professional,Administrative and support,Public administration and defence,Education,Human health,Arts $ entertainment,Other services,Activities of households,Extraterritorial organizations
0,Kenya,15-24,female,2015,4742700.0,55.928,2.652497e+06,7.339,153399.979382,1.936803e+06,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0,0.000000e+00,0
1,Kenya,15-24,female,2016,4887980.0,56.361,2.754914e+06,7.371,157228.264801,1.975837e+06,...,485418.609963,848269.122880,2.300956e+06,3.555035e+06,1.231482e+07,3.942594e+06,0,0,9.490859e+06,0
2,Kenya,15-24,female,2017,5033260.0,56.906,2.864227e+06,9.304,201806.836312,1.967226e+06,...,473589.250578,827597.314871,2.244883e+06,3.570008e+06,1.241247e+07,3.957306e+06,0,0,9.601153e+06,0
3,Kenya,15-24,female,2018,5178540.0,57.463,2.975744e+06,11.192,246536.879053,1.956259e+06,...,490568.613533,857268.755201,2.325368e+06,3.617842e+06,1.249134e+07,3.990789e+06,0,0,9.598457e+06,0
4,Kenya,15-24,female,2019,5323820.0,58.043,3.090105e+06,13.175,294291.971987,1.939423e+06,...,5175.457976,9044.113900,2.453244e+04,3.655088e+04,1.292827e+05,4.011513e+04,0,0,9.543569e+04,0


#### 7. Analyze Employment, Unemployment, and Inactivity by Education:
We filter the data to include only the relevant education levels then calculate rates based on shares and multiply by the employed or inactive population as appropriate.
 

In [174]:
education_df = pop_df[['name',	'age_group','sex','year','population','total_inactive_population','total_unemployed_population','total_employed_population']]

education_df.head()

,name,age_group,sex,year,population,total_inactive_population,total_unemployed_population,total_employed_population
0,Kenya,15-24,female,2015,4742700.0,2.652497e+06,153399.979382,1.936803e+06
1,Kenya,15-24,female,2016,4887980.0,2.754914e+06,157228.264801,1.975837e+06
2,Kenya,15-24,female,2017,5033260.0,2.864227e+06,201806.836312,1.967226e+06
3,Kenya,15-24,female,2018,5178540.0,2.975744e+06,246536.879053,1.956259e+06
4,Kenya,15-24,female,2019,5323820.0,3.090105e+06,294291.971987,1.939423e+06


In [175]:
def process_inactivity_data(df, years_to_filter):
    # Filter by indicator and relevant columns
    inactivity_df = df[df['indicator.label'] == 'Inactivity rate by sex, age and education (%)']
    inactivity_df = inactivity_df[inactivity_df['time'].isin(years_to_filter)]
    inactivity_df = inactivity_df[inactivity_df['classif1.label'].isin(['Age (Youth, adults): 15-24', 'Age (Youth, adults): 25+'])]
    
    # Map age groups to common labels
    inactivity_df['classif1.label'] = inactivity_df['classif1.label'].str.replace('Age (Youth, adults): 25+', '25-35', regex=False).str.strip()
    inactivity_df['classif1.label'] = inactivity_df['classif1.label'].str.replace('Age (Youth, adults): 15-24', '15-24', regex=False).str.strip()
    
    # Filter by sex labels
    inactivity_df = inactivity_df[inactivity_df['sex.label'].isin(['Sex: Male', 'Sex: Female'])]
    
    # Map sex labels to common labels
    inactivity_df['sex.label'] = inactivity_df['sex.label'].str.replace('Sex: Male', 'male', regex=False).str.strip()
    inactivity_df['sex.label'] = inactivity_df['sex.label'].str.replace('Sex: Female', 'female', regex=False).str.strip()
    
    # Filter by classif2.label
    inactivity_df = inactivity_df[inactivity_df['classif2.label'].str.contains('Aggregate levels')]
    
    return inactivity_df
 # Replace with your desired years
filtered_inactivity_df = process_inactivity_data(kenyan_df, years_to_filter)
filtered_inactivity_df.head()


,ref_area,ref_area.label,source,source.label,indicator,indicator.label,sex,sex.label,classif1,classif1.label,...,time,obs_value,obs_status,obs_status.label,note_classif,note_classif.label,note_indicator,note_indicator.label,note_source,note_source.label
8907,KEN,Kenya,BB:7021,HIES - Household Budget Survey,EIP_DWAP_SEX_AGE_EDU_RT,"Inactivity rate by sex, age and education (%)",SEX_M,male,AGE_YTHADULT_Y15-24,15-24,...,2019,59.77,NaN,NaN,NaN,NaN,NaN,NaN,R1:3513,Repository: ILO-STATISTICS - Micro data proces...
8908,KEN,Kenya,BB:7021,HIES - Household Budget Survey,EIP_DWAP_SEX_AGE_EDU_RT,"Inactivity rate by sex, age and education (%)",SEX_M,male,AGE_YTHADULT_Y15-24,15-24,...,2019,54.49,NaN,NaN,NaN,NaN,NaN,NaN,R1:3513,Repository: ILO-STATISTICS - Micro data proces...
8909,KEN,Kenya,BB:7021,HIES - Household Budget Survey,EIP_DWAP_SEX_AGE_EDU_RT,"Inactivity rate by sex, age and education (%)",SEX_M,male,AGE_YTHADULT_Y15-24,15-24,...,2019,66.21,NaN,NaN,NaN,NaN,NaN,NaN,R1:3513,Repository: ILO-STATISTICS - Micro data proces...
8910,KEN,Kenya,BB:7021,HIES - Household Budget Survey,EIP_DWAP_SEX_AGE_EDU_RT,"Inactivity rate by sex, age and education (%)",SEX_M,male,AGE_YTHADULT_Y15-24,15-24,...,2019,46.86,NaN,NaN,NaN,NaN,NaN,NaN,R1:3513,Repository: ILO-STATISTICS - Micro data proces...
8911,KEN,Kenya,BB:7021,HIES - Household Budget Survey,EIP_DWAP_SEX_AGE_EDU_RT,"Inactivity rate by sex, age and education (%)",SEX_M,male,AGE_YTHADULT_Y15-24,15-24,...,2019,23.63,NaN,NaN,NaN,NaN,NaN,NaN,R1:3513,Repository: ILO-STATISTICS - Micro data proces...


In [176]:
def process_education_inactivity_data(inactivity_df, education_levels):
    # Define the suffix for new column names
    suffix = '_inactive'
    # Generate new column names with the suffix for each education level
    new_column_names = [level + suffix for level in education_levels]
    # Create a new DataFrame 'education_df' by assigning new columns with zeros
    education_df = inactivity_df.assign(**{column: 0 for column in new_column_names})

    # Create a dictionary mapping a combination of attributes to observed values
    obs_value_map = inactivity_df.set_index(['classif2.label', 'classif1.label', 'time', 'sex.label'])['obs_value'].to_dict()

    # Prepare data for the new DataFrame by iterating through the obs_value_map
    edu_levels_data = []
    for key, val in obs_value_map.items():
        # Extract relevant attributes and convert the observed value to numeric
        edu_levels_data.append((*key[:3], key[3], pd.to_numeric(val, errors='coerce')))

    # Create and return a new DataFrame using the prepared data and column names
    return pd.DataFrame(edu_levels_data, columns=['education_level', 'age_group', 'year', 'sex', 'inactivity_rate'])

# Define the education levels for which inactivity rates are being processed
education_levels = ['Less than basic', 'Basic', 'Intermediate', 'Advanced', 'Level not stated']

# Process inactivity data using the defined function and education levels
edu_levels_df = process_education_inactivity_data(filtered_inactivity_df, education_levels)

# Display the first few rows of the processed DataFrame
edu_levels_df.head()


,education_level,age_group,year,sex,inactivity_rate
0,Education (Aggregate levels): Total,15-24,2019,male,59.77
1,Education (Aggregate levels): Less than basic,15-24,2019,male,54.49
2,Education (Aggregate levels): Basic,15-24,2019,male,66.21
3,Education (Aggregate levels): Intermediate,15-24,2019,male,46.86
4,Education (Aggregate levels): Advanced,15-24,2019,male,23.63


In [177]:
def impute_inactivity_rates(education_df, edu_levels_df, education_levels, age_groups):
    # Iterate through each row in the education_df dataframe
    for index, row in education_df.iterrows():
        for level in education_levels:
            for age_group in age_groups:
                if row['year'] in [2016, 2019]:
                    condition = (edu_levels_df['education_level'].str.contains(level)) & \
                                (edu_levels_df['sex'] == row['sex']) & \
                                (edu_levels_df['age_group'] == row['age_group']) & \
                                (edu_levels_df['year'] == row['year'])

                    # Get the inactivity rate from the edu_levels_df based on the conditions
                    inactivity_rate_row = edu_levels_df[condition]

                    # Check if inactivity_rate_row is not empty
                    if not inactivity_rate_row.empty:
                        inactivity_rate = inactivity_rate_row['inactivity_rate'].values[0]
                        education_df.at[index, level + '_inactive'] = inactivity_rate

                if row['year'] > 2019:
                    condition = (edu_levels_df['education_level'].str.contains(level)) & \
                                (edu_levels_df['sex'] == row['sex']) & \
                                (edu_levels_df['age_group'] == row['age_group']) & \
                                (edu_levels_df['year'] == 2019)

                    # Get the inactivity rate for the year 2019 from the edu_levels_df
                    inactivity_rate_row = edu_levels_df[condition]

                    # Check if inactivity_rate_row is not empty
                    if not inactivity_rate_row.empty:
                        inactivity_rate = inactivity_rate_row['inactivity_rate'].values[0]
                        total_inactive_population = education_df.at[index, 'total_inactive_population']
                        education_df.at[index, level + '_inactive'] = inactivity_rate * total_inactive_population

                else:
                    # Handle the case when inactivity_rate_row is empty (no matching data found)
                    print(f"No inactivity rate data found for '{level}' and year {row['year']}")

# Define education levels and age groups
education_levels = ['Less than basic', 'Basic', 'Intermediate', 'Advanced', 'Level not stated']
age_groups = ['15-24', '25-35']

# Call the function to impute inactivity rates
impute_inactivity_rates(education_df, edu_levels_df, education_levels, age_groups)


No inactivity rate data found for 'Less than basic' and year 2015
No inactivity rate data found for 'Less than basic' and year 2015
No inactivity rate data found for 'Basic' and year 2015
No inactivity rate data found for 'Basic' and year 2015
No inactivity rate data found for 'Intermediate' and year 2015
No inactivity rate data found for 'Intermediate' and year 2015
No inactivity rate data found for 'Advanced' and year 2015
No inactivity rate data found for 'Advanced' and year 2015
No inactivity rate data found for 'Level not stated' and year 2015
No inactivity rate data found for 'Level not stated' and year 2015
No inactivity rate data found for 'Less than basic' and year 2016
No inactivity rate data found for 'Less than basic' and year 2016
No inactivity rate data found for 'Basic' and year 2016
No inactivity rate data found for 'Basic' and year 2016
No inactivity rate data found for 'Intermediate' and year 2016
No inactivity rate data found for 'Intermediate' and year 2016
No inact

#### 8. Unemployed
- Analyze Trends and Differences:
Use the merged data to analyze trends in the labor market, education levels, and sectors. Visualize and interpret the results.


In [178]:
def process_unemployment_data(kenya_df, years_to_filter):
    # Step 1: Filter by indicator label
    unemployment_sex_age_education = kenya_df[kenya_df['indicator.label'] == 'Employment by sex, age and education (thousands)']

    # Step 2: Filter by years
    unemployment_sex_age_education = unemployment_sex_age_education[unemployment_sex_age_education['time'].isin(years_to_filter)]

    # Step 3: Filter by classif1.label (age groups)
    age_group_mapping = {
        'Age (Youth, adults): 15-24': '15-24',
        'Age (Youth, adults): 25+': '25-35'
    }
    unemployment_sex_age_education['classif1.label'] = unemployment_sex_age_education['classif1.label'].replace(age_group_mapping, regex=False).str.strip()
    unemployment_sex_age_education = unemployment_sex_age_education[unemployment_sex_age_education['classif1.label'].isin(['15-24', '25-35'])]

    # Step 4: Modify sex labels
    sex_mapping = {
        'Sex: Male': 'male',
        'Sex: Female': 'female'
    }
    unemployment_sex_age_education['sex.label'] = unemployment_sex_age_education['sex.label'].replace(sex_mapping, regex=False).str.strip()

    # Step 5: Filter by sex.label
    unemployment_sex_age_education = unemployment_sex_age_education[unemployment_sex_age_education['sex.label'].isin(['male', 'female'])]

    # Step 6: Filter by classif2.label (aggregate levels)
    unemployment_sex_age_education = unemployment_sex_age_education[unemployment_sex_age_education['classif2.label'].str.contains('Aggregate levels')]

    # Group by specified columns and sum obs_value
    unemployment_sex_age_education = unemployment_sex_age_education.groupby(['classif2.label', 'classif1.label', 'sex.label', 'time'])['obs_value'].sum().reset_index()
    
    # Convert 'obs_value' column to numeric
    unemployment_sex_age_education['obs_value'] = pd.to_numeric(unemployment_sex_age_education['obs_value'], errors='coerce')
    
    # Drop any remaining invalid values
    unemployment_sex_age_education = unemployment_sex_age_education.dropna(subset=['obs_value'])

    return unemployment_sex_age_education

# Usage
unemployment_sex_age_education = process_unemployment_data(kenyan_df, years_to_filter)
unemployment_sex_age_education.head()


,classif2.label,classif1.label,sex.label,time,obs_value
0,Education (Aggregate levels): Advanced,15-24,female,2016,62.565
1,Education (Aggregate levels): Advanced,15-24,female,2019,25.181
2,Education (Aggregate levels): Advanced,15-24,male,2016,58.715
3,Education (Aggregate levels): Advanced,15-24,male,2019,25.810
4,Education (Aggregate levels): Advanced,25-35,female,2016,231.202


In [179]:
def calculate_and_merge_unemployment_shares(unemployment_sex_age_education):
    # Calculate total values for each year, sex, level, and age_group
    total_values = unemployment_sex_age_education.groupby(["sex.label", "classif2.label", "classif1.label", "time"])["obs_value"].sum().reset_index()

    # Iterate through unique levels, years, genders, and age groups
    for level in unemployment_sex_age_education["classif2.label"].unique():
        level_rows = unemployment_sex_age_education[unemployment_sex_age_education["classif2.label"] == level]
        for year in unemployment_sex_age_education["time"].unique():
            for gender in unemployment_sex_age_education["sex.label"].unique():
                for age_group in unemployment_sex_age_education["classif1.label"].unique():
                   # Filter total_values based on the current year, gender, level, and age_group
                    level_total = total_values[
                        (total_values["time"] == year) & 
                        (total_values["sex.label"] == gender) & 
                        (total_values["classif1.label"] == age_group)
                    ]["obs_value"].values[0]
                    
                    # Filter rows based on the current year, gender, level, and age_group
                    year_gender_age_rows = level_rows[
                        (level_rows["time"] == year) & 
                        (level_rows["sex.label"] == gender) & 
                        (level_rows["classif1.label"] == age_group)
                    ]
                    
                    # Calculate and update sector shares
                    unemployment_sex_age_education.loc[year_gender_age_rows.index, "level_shares"] = year_gender_age_rows["obs_value"] / level_total
    
    return unemployment_sex_age_education

# Usage
unemployment_sex_age_education = calculate_and_merge_unemployment_shares(unemployment_sex_age_education)
unemployment_sex_age_education.head()


,classif2.label,classif1.label,sex.label,time,obs_value,level_shares
0,Education (Aggregate levels): Advanced,15-24,female,2016,62.565,1.0
1,Education (Aggregate levels): Advanced,15-24,female,2019,25.181,1.0
2,Education (Aggregate levels): Advanced,15-24,male,2016,58.715,1.0
3,Education (Aggregate levels): Advanced,15-24,male,2019,25.810,1.0
4,Education (Aggregate levels): Advanced,25-35,female,2016,231.202,1.0


In [180]:
def calculate_and_merge_level_shares(df):
    """
    Calculate and merge level shares based on total values.

    Parameters:
    df (DataFrame): DataFrame containing unemployment data by sex, age, and education.

    Returns:
    DataFrame: Updated DataFrame with calculated level shares.
    """
    # Calculate total values for each year, sex, level, and age_group
    total_values = df[unemployment_sex_age_education["classif2.label"].str.contains("Total")]
    total_values = total_values.groupby(["sex.label", "classif2.label", "classif1.label", "time"])["obs_value"].sum().reset_index()

    # Iterate through unique levels, years, genders, and age groups
    for level in df["classif2.label"].unique():
        level_rows = df[df["classif2.label"] == level]
        for year in df["time"].unique():
            for gender in df["sex.label"].unique():
                for age_group in df["classif1.label"].unique():
                    # Filter total_values based on the current year, gender, level, and age_group
                    level_total = total_values[
                        (total_values["time"] == year) & 
                        (total_values["sex.label"] == gender) & 
                        (total_values["classif1.label"] == age_group)
                    ]["obs_value"].values[0]
                    
                    # Filter rows based on the current year, gender, level, and age_group
                    year_gender_age_rows = level_rows[
                        (level_rows["time"] == year) & 
                        (level_rows["sex.label"] == gender) & 
                        (level_rows["classif1.label"] == age_group)
                    ]
                    
                    # Calculate and update level shares
                    df.loc[year_gender_age_rows.index, "level_shares"] = year_gender_age_rows["obs_value"] / level_total
    
    return df

# Usage
unemployment_sex_age_education = calculate_and_merge_level_shares(unemployment_sex_age_education)
unemployment_sex_age_education.head()


,classif2.label,classif1.label,sex.label,time,obs_value,level_shares
0,Education (Aggregate levels): Advanced,15-24,female,2016,62.565,0.034158
1,Education (Aggregate levels): Advanced,15-24,female,2019,25.181,0.017975
2,Education (Aggregate levels): Advanced,15-24,male,2016,58.715,0.029910
3,Education (Aggregate levels): Advanced,15-24,male,2019,25.810,0.015993
4,Education (Aggregate levels): Advanced,25-35,female,2016,231.202,0.030205


In [181]:
def calculate_and_merge_unemployment_rates(df, education_levels, df2):
    """
    Calculate and merge unemployment rates based on level shares and total inactive population.

    Parameters:
    education_df (DataFrame): DataFrame containing education and inactivity data.
    education_levels (list): List of education levels.
    unemployment_sex_age_education (DataFrame): DataFrame containing unemployment data by sex, age, and education.

    Returns:
    DataFrame: Updated education DataFrame with calculated unemployment rates.
    """
    age_groups = ['15-24', '25-35']

    for index, row in df.iterrows():
        for level in education_levels:
            for age_group in age_groups:
                if row['year'] in [2016, 2019]:
                    unemployment_rate_row = df2[
                        (df2['classif2.label'].str.contains(level)) &
                        (df2['sex.label'] == row['sex']) &
                        (df2['classif1.label'] == row['age_group']) &
                        (df2['time'] == row['year'])
                    ]

                    if not unemployment_rate_row.empty:
                        unemployment_rate = unemployment_rate_row['level_shares'].values[0]
                        df.at[index, level + '_unemployment'] = unemployment_rate * education_df.at[index, 'total_inactive_population']

                if row['year'] > 2019:
                    unemployment_rate_row = df2[
                        (df2['classif2.label'].str.contains(level)) &
                        (df2['sex.label'] == row['sex']) &
                        (df2['classif1.label'] == row['age_group']) &
                        (df2['time'] == row['year'])
                    ]

                    if not unemployment_rate_row.empty:
                        unemployment_rate = unemployment_rate_row['level_shares'].values[0]
                        df.at[index, level + '_unemployment'] = unemployment_rate * df.at[index, 'total_inactive_population']

    return df

# Usage
updated_education_df = calculate_and_merge_unemployment_rates(education_df, education_levels, unemployment_sex_age_education)


After all these imputations, we now save our imputed dataframes as csv then which will then use for further analysis

*NB* **The functions defined are could be used for the other other countries in our original datasets**

In [182]:
education_df
education_df = education_df

# Save the DataFrame as a CSV file
education_df.to_csv('education_kenya.csv', index=False)

In [183]:

pop_df
pop_df = pop_df

# Save the DataFrame as a CSV file
pop_df.to_csv('pop_kenya.csv', index=False)

In [184]:
education_df.head()

,name,age_group,sex,year,population,total_inactive_population,total_unemployed_population,total_employed_population,Less than basic_inactive,Basic_inactive,Intermediate_inactive,Advanced_inactive,Level not stated_inactive,Less than basic_unemployment,Basic_unemployment,Intermediate_unemployment,Advanced_unemployment,Level not stated_unemployment
0,Kenya,15-24,female,2015,4742700.0,2.652497e+06,153399.979382,1.936803e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Kenya,15-24,female,2016,4887980.0,2.754914e+06,157228.264801,1.975837e+06,47.88,51.74,60.87,60.76,67.41,133251.123331,1.321644e+06,1.198570e+06,94101.818760,7347.356903
2,Kenya,15-24,female,2017,5033260.0,2.864227e+06,201806.836312,1.967226e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Kenya,15-24,female,2018,5178540.0,2.975744e+06,246536.879053,1.956259e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Kenya,15-24,female,2019,5323820.0,3.090105e+06,294291.971987,1.939423e+06,68.38,68.03,56.99,31.24,47.56,246619.604416,1.992582e+06,7.778941e+05,55545.273909,17461.434743
